<a href="https://colab.research.google.com/github/LuisCastroV/ADA-Informes/blob/main/Informe_12_Bellman_Ford.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Problema del camino más corto con fuente única *(Algoritmo Bellman-Ford)*
---
**Entrada**: Un grafo dirigido $G=(V,E)$, un vértice fuente $s\in V$, y un valor real $l_e$ asociado a cada arco $e\in E$.

**Salida**: Una de las siguientes opciones:

- La distancia más corta $dist(s,v)$ para cada vértice $v\in V$.
- Una declaración indicando que $G$ contiene un ciclo negativo.

---
**Imagen referencial del Algoritmo Bellman-Ford:**
![image](https://static.javatpoint.com/tutorial/daa/images/bellman-ford-algorithm.png)

---
**Breve información:** El algoritmo de Bellman-Ford nos ayuda a encontrar el camino más corto desde un vértice a todos los demás vértices de un grafo ponderado, es similar al algoritmo de Dijkstra pero puede funcionar con grafos en los que los arista pueden tener pesos negativos.

---


# 2. Descripción del algoritmo

##2.1. Bellman-Ford

Consiste en un grafo dirigido $G$ con $n$ vértices, donde cada arco posee un peso asignado (distancia), más un nodo $s$ que corresponde al punto de partida. Si no existen ciclos negativos, el algoritmo retorna una lista con la distancia mínima que existe entre el nodo inicial y el resto de nodos del grafo. En caso contrario, el algoritmo retorna una lista vacía. Los pasos realizados son los siguientes:

- Se crea una lista para guardar la distancia mínima de $s$ al resto de nodos, inicializando sus valores en infinito, asignamos al nodo $s$ una distancia de 0, puesto que corresponde al nodo inicial.

- Iteramos $V$$-$$1$ veces por todos los arcos del grafo o hasta que no existan más cambios en las distancias (lo que ocurra primero).

- Para cada arco $(u,v)$, calculamos la distancia de $s$ a $v$ como $dist(s,v) = dist(s,u)+w(u,v)$ , donde $w(u,v)$ corresponde al peso del arco $(u,v)$.

- Si la distancia calculada en el paso anterior es menor a la actual, actualizamos su valor.

- Al finalizar las iteraciones, realizamos una última iteración adicional para verificar que no existan ciclos negativos. Si para cualquier arco $(u,v)$ obtenemos una distancia menor a las previamente calculadas, retornamos una lista vacía.

- Si no existen ciclos negativos, retornamos la lista con las distancias obtenidas.

In [85]:
from timeit import repeat
from math import log
from random import randint
import matplotlib.pyplot as plt
import networkx as nx
import random
import sys

%matplotlib inline

def is_valid_edge(generated_edges: dict, i: int, j: int):
    return i != j and not generated_edges.get((i, j), None) and not generated_edges.get((j, i), None)

def instance_generator(n: int):
    graph = []
    nodes = random.sample(range(0, n), n)
    unvisited_nodes = random.sample(range(0, n), n)
    generated_edges = {}
    for i in nodes:
        rand = random.sample(nodes, random.randint(1, 3))

        for j in rand:
            edge = (i, j)
            edge_with_weight = (i, j, random.randint(1, 50))
            if generated_edges.get((edge[1], edge[0]), None):
                continue
            if i == j:
                new_vertice = None
                iterations = 0
                while new_vertice is None and iterations < 250:
                    iterations += 1
                    number = random.randint(0, n - 1)
                    if is_valid_edge(generated_edges, i, number):
                        new_vertice = number
                if iterations >= 250:
                    return instance_generator(n)
                edge = (i, new_vertice)
                edge_with_weight = (i, new_vertice, random.randint(-50, 50)) 
            graph.append(edge_with_weight)
            generated_edges[edge] = edge
            if edge_with_weight[1] in unvisited_nodes:
                unvisited_nodes.remove(edge_with_weight[1])
    for i in unvisited_nodes:
        valid_edge = False
        iterations = 0
        while not valid_edge and iterations < 250:
            iterations += 1
            m = random.randint(0, n - 1)
            if is_valid_edge(generated_edges, m, i):
                valid_edge = True
                edge = (m, i)
                edge_with_weight = (m, i, random.randint(-50, 50))
                graph.append(edge_with_weight)
                generated_edges[edge] = edge
        if iterations >= 250:
            return instance_generator(n)
    return graph, graph[0][0]




In [92]:
class GraphBF():
    def __init__(self, vertices):
        self.V = vertices  
        self.graph = list()

    def addEdge(self, u, v, w):
        self.graph.append((u, v, w))

    def printArr(self, dist):
        print("Distancia del vértice desde la fuente")
        for i in range(self.V):
            print("{0}\t\t{1}".format(i, dist[i]))
    def BellmanFord(self, src,print_solution=True):
        distancia = [float("Inf")] * self.V
        distancia[src] = 0
        for i in range(self.V - 1):
            for a, b, c in self.graph:
                if distancia[a] != float("Inf") and distancia[a] + c < distancia[b]:
                    distancia[b] = distancia[a] + c
        for a, b, c in self.graph:
            if distancia[a] != float("Inf") and distancia[a] + c < distancia[b]:
                if print_solution:
                    print("El grafo contiene un ciclo negativo")
                distancia = list()
                return distancia
        if print_solution:
            print("No existen ciclos negativos\n")
            self.printArr(distancia)
            
n = 6
g, root = instance_generator(n)
grafo = GraphBF(n)
for (x, y, peso) in g:
    grafo.addEdge(x, y, peso)
grafo.BellmanFord(root)

No existen ciclos negativos

Distancia del vértice desde la fuente
0		-50
1		0
2		25
3		-10
4		25
5		-25


##2.2. Dijkstra

- Se crea un conjunto sptSet (conjunto de árboles de ruta más corta) que realiza un seguimiento de los vértices incluidos en el árbol de ruta más corta, es decir, cuya distancia mínima desde la fuente se calcula y finaliza. Inicialmente, este conjunto está vacío.
- Se asigna un valor de distancia a todos los vértices en el gráfico de entrada. Se inicializan todos los valores de distancia como $infinitos$, luego se le asigna el valor de distancia como $0$ al vértice de origen para que se elija primero.
- Mientras sptSet no incluye todos los vértices:
 - Se elige un vértice $u$ que no esté en sptSet y tenga un valor de distancia mínimo.
 - Se incluye $u$ en sptSet.
 - Se actualiza el valor de distancia de todos los vértices adyacentes de $u$. Para actualizar los valores de distancia, se itera a través de todos los vértices adyacentes. 
   - Para cada vértice adyacente $v$, si la suma de un valor de distancia de $u$ *(desde la fuente)* y el peso del borde $u-v$ es menor que el valor de distancia de $v$, entonces se actualiza el valor de distancia de $v$.

In [93]:
class Graph():
    def __init__(self, vertices):
        self.V = vertices
        self.graph = [[0 for column in range(vertices)]
                      for row in range(vertices)]

    def printSolution(self, dist):
        print("Vertex \t Distance from Source")
        for node in range(self.V):
            print(node, "\t\t", dist[node])
 
    def minDistance(self, dist, sptSet):
        min = 1e7
        for v in range(self.V):
            if dist[v] < min and sptSet[v] == False:
                min = dist[v]
                min_index = v
        return min_index

    def dijkstra(self, src):
        dist = [1e7] * self.V
        dist[src] = 0
        sptSet = [False] * self.V
 
        for cout in range(self.V):
            u = self.minDistance(dist, sptSet)
            sptSet[u] = True
            for v in range(self.V):
                if (self.graph[u][v] > 0 and
                   sptSet[v] == False and
                   dist[v] > dist[u] + self.graph[u][v]):
                    dist[v] = dist[u] + self.graph[u][v]
        self.printSolution(dist)
 
g = Graph(9)
g.graph = [[0, 4, 0, 0, 0, 0, 0, 8, 0],
           [4, 0, 8, 0, 0, 0, 0, 11, 0],
           [0, 8, 0, 7, 0, 4, 0, 0, 2],
           [0, 0, 7, 0, 9, 14, 0, 0, 0],
           [0, 0, 0, 9, 0, 10, 0, 0, 0],
           [0, 0, 4, 14, 10, 0, 2, 0, 0],
           [0, 0, 0, 0, 0, 2, 0, 1, 6],
           [8, 11, 0, 0, 0, 0, 1, 0, 7],
           [0, 0, 2, 0, 0, 0, 6, 7, 0]
           ]
 
g.dijkstra(0)
 

Vertex 	 Distance from Source
0 		 0
1 		 4
2 		 12
3 		 19
4 		 21
5 		 11
6 		 9
7 		 8
8 		 14


#2.3 Ejemplo paso a paso

**Paso 1:** Los pesos estan en color azul y la distancia inicial en cada vértice es infinito.

![image](https://jariasf.files.wordpress.com/2012/03/grafo.jpg)

![image](https://jariasf.files.wordpress.com/2013/01/tablabellman1.jpg?w=768&h=89)

**Paso 2:** Inicialmente la distancia de vértice 1 -> vértice 1 es $0$ por estar en el mismo lugar.

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman1.jpg)

**Paso 3:** Se empieza con las aristas que parten del vértice 1, se observa que tenemos 2 aristas, la que une vértice 1 con vértice 2 y vértice 1 con vértice 4.

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman2.jpg)

**Paso 4:** La distancia actual desde el vértice inicial a 2 es $∞$, verifiquemos el paso de relajación: 

distancia[1]+7 < distancia[2] -> $0+7 < ∞$  ->   $7 < ∞$. 

El paso de relajación es posible realizarlo entonces actualizamos la distancia en el vértice 2.

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman3.jpg)

**Paso 5:** El paso de relajación es posible realizarlo entonces actualizamos la distancia en el vértice 2.

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman4.jpg)

**Paso 6:** Ahora se evalua al siguiente adyacente que es el vértice 4.

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman5.jpg)

**Paso 7:** Sucede lo mismo que el paso 4.

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman6.jpg)

**Paso 8:** Terminadas las aristas que parten del vértice 1, continuamos con las aristas que parten del vértice 2.

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman7.jpg)

**Paso 9:**

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman8.jpg)

**Paso 10:** En este caso vemos que no se lleva acabo el paso de relajación:

distancia[2]+2< distancia[4] -> $7 + 2 < 2$  -> $9 < 2$

Por lo tanto no actualizamos valores en la tabla. Ahora el siguiente vértice a evaluar es el vértice 3 que posee una sola arista, y asi se iran "repitiendo" los pasos siguientes.

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman9.jpg)

**Paso 11:**

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman10jpg.jpg)

**Paso 12:**

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman11.jpg)

**Paso 13:**

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman12.jpg)

**Paso 14:**

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman13.jpg)

**Paso 15:**

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman14.jpg)

**Paso 16:**

![image](https://jariasf.files.wordpress.com/2013/01/grafobellman15.jpg)

![image](https://jariasf.files.wordpress.com/2013/01/tablabellman8.jpg?w=768)

#3. Correctitud

Para poder probar que el algoritmo de Bellman-Ford funciona, utilizaremos *inducción matemática*, donde:

- Probar $P(n)$  para un caso base, por ejemplo  $P(1)$.
- Probar que si $P(m)$ es cierto, donde  $m < n$, entonces  $P(n)$ también lo es.


$Teorema$: Sea G un grafo (orientado o no) sin circuitos negativos y  s∈V  un nodo de origen. Al comenzar la k-ésima iteración, el algoritmo de Bellman-Ford determina un camino mínimo de a lo más k − 1 aristas de  s  a los nodos de G.

$Caso base$: ( k=1 ) Los camino mínimos de longitud cero desde  s  son los que van a  s  y a los nodos no alcanzables desde  s . Luego el lema vale trivialmente por cómo se inicializaron los valores de ϵ.

$Paso inductivo$: Supongamos que el lema vale para algún  k≥1  y veamos que vale para  k+1 . Sea el comienzo de la k-ésima iteración y sea  vϵV . Si existe un camino mínimo de a lo más  k−1  aristas a  v , por hipótesis inductiva, el algoritmo ya lo determinó. Supongamos ahora que los camino mínimo a  v  tienen exactamente  k  aristas y sea  P  uno de ellos y  (u,v)  la última arista de  P . Por subestructura óptim de camino mínimo,  P′=P−(u,v)  es un camino mínimo a  u . Luego, por hipótesis inductiva,  ϵ(u)=𝝳(s,u)+w(u,v)=w(P)=𝝳(s,v) .

#4. Tiempo de ejecución 

#4.1. Tiempo de ejecución de Bellman-Ford

- En primer lugar, se realiza el paso de inicialización ⇒ O(V)

- Luego, el algoritmo itera  |V|−1  los tiempos con cada iteración ⇒ O(1)

- Después de  |V|−1  de las interacciones. el algoritmo elige todas las aristas y luego llama la función Relax(). Elegir todos los bordes lleva tiempo más la función Relax() ⇒ O(1) + O(E)

En conclusión, la complejidad para hacer todas la operaciones del algoritmo lleva un tiempo de O(VE)

#4.2. Tiempo de ejecución de Dijkstra
Orden de complejidad del algoritmo:

O(|V|²+|A|)=O(|V|²) , sin utilizar cola de prioridad,  
O((|A|+|V|)log|V|)=O(|A|log|V|) utilizando cola de prioridad (por ejemplo, un montículo binario o un árbol binario balanceado). Por otro lado, si se utiliza un montículo de Fibonacci, sería  O(|V|log|V|+|A|).

La complejidad de este algoritmo se puede calcular contando las operaciones realizadas:

- El algoritmo consiste en  n−1  iteraciones, como máximo. En cada iteración, se añade un vértice al conjunto distinguido.

- En cada iteración, se identifica el vérticde con la menor etiqueta entre los que no están en  Sk . El número de estas operaciones está acotado por  n−1 .

- Además, se realizan una suma y una comparación para actualizar la etiqueta de cada uno de los vértices que no están en  Sk .

Luego, en cada iteración se realizan a lo más  2(n−1)  operaciones. Entonces:

$Teorema$: El algoritmo de Dijkstra realiza O(n²) operaciones (sumas y comparaciones) para determinar la longitud del camino más corto entre dos vértices de un grafo ponderado simple, conexo y no dirigido con n vértices.

En general:

Tiempo de ejecución =  O(|A|∗Tdk+|v|∗Tdm) 

|A| : Número de aristas

Tdk : Complejidad de disminuir clave

|V| : Número de vértices

Tdm : Complejidad de extraer mínimo

#5. Experimento

El algoritmo de arriba representa el Dijkstra y el de abajo al algoritmo de Bellman-Ford.

![image](https://upload.wikimedia.org/wikipedia/commons/2/2e/Shortest_path_Dijkstra_vs_BellmanFord.gif)


#5.1. Djikstra vs Bellman-Ford

![image](https://www.happycoders.eu/wp-content/uploads/2021/03/bellman-ford-vs-dijkstra.png)

*Esta grafica fue extraida de Internet*, se puede observa que a medida que crece el tamaño del grafo el algoritmo de Bellman-Ford tanto estándar como optimizado empieza a aumentar de forma cuadrática en su tiempo de ejecución en comparación al algoritmo de Djikstra implementado con lista de prioiridad y Fibonacci.